In [6]:
import os
import openai
import sys
import json
sys.path.append('../..')

In [7]:
openai.api_key = os.environ['OPENAI_API_KEY']

In [8]:
# from langchain.prompts.chat import (
#     ChatPromptTemplate,
#     HumanMessagePromptTemplate,
#     SystemMessagePromptTemplate,
# )
# from langchain.schema import HumanMessage, SystemMessage
# from langchain_openai import ChatOpenAI
# chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [9]:
# messages = [
#     SystemMessage(
#         content="You are a helpful assistant that translates English to French."
#     ),
#     HumanMessage(
#         content="Translate this sentence from English to Pirate. I love programming."
#     ),
# ]
# chat.invoke(messages)

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
# model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1)
# output_parser = StrOutputParser()

# chain = prompt | model | output_parser

# chain.invoke({"topic": "ice cream"})

In [60]:
import random
def design_logo_concepts(topic: str):
    model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    template = """Answer yes if the user will need a logo, otherwise, answer no. Here is the user prompt

    Prompt = {prompt}"""
    
    prompt = ChatPromptTemplate.from_template(template)
    output_parser = StrOutputParser()
    inner_chain = prompt | model | output_parser
    if 'Yes' in inner_chain.invoke({"prompt" : topic}):
        index = random.randint(1,4)
        s3_path = f's3://designs/logo_{index}.jpg'
        return f'{{ component: \"Image\", props: {{ url: {s3_path}, }} }}'
    return 'NONE'

In [61]:
from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnableLambda
from langchain.schema.runnable import RunnableMap
from langchain_openai import ChatOpenAI

template = """If KEY is 'NONE', then you should kindly respond to the prompt in a conversation. Otherwise, if KEY is not 'NONE', 
you will answer with the given KEY

prompt: {prompt}
KEY: {response}

"""

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
output_parser = StrOutputParser()

chain = RunnableMap({
    "response": lambda x : design_logo_concepts(x["prompt"]),
    "prompt": lambda x : x["prompt"]
}) | prompt | model | output_parser

In [62]:
def interact_with_model(input):
    output = chain.invoke({"prompt": input})
    try:
        return json.loads(output)
    except:
        return output

In [63]:
interact_with_model("hi")

'Hi, how can I assist you today?'

In [52]:
interact_with_model("I think ice cream is great")

'I agree, ice cream is indeed great!'

In [64]:
interact_with_model("I would like an ice cream truck logo")

{'component': 'Image', 'props': {'url': 's3://designs/logo_1.jpg'}}

In [54]:
interact_with_model("bye bye")

'Goodbye!'

In [65]:
interact_with_model("NOOOO, I need another logo, perhaps a logo for my UI task")

'{ component: "Image", props: { url: s3://designs/logo_2.jpg, } }'

In [66]:
interact_with_model("an image would be nice, I need one.")

{'component': 'Image', 'props': {'url': 's3://designs/logo_4.jpg'}}

In [68]:
interact_with_model("tell me about cornell")

"Cornell University is a prestigious Ivy League institution located in Ithaca, New York. It was founded in 1865 and is known for its exceptional academic programs, diverse student body, and beautiful campus. Cornell offers a wide range of undergraduate and graduate programs across various fields of study, including engineering, business, agriculture, arts and sciences, and more. The university is also renowned for its research initiatives and has produced numerous notable alumni, including Nobel laureates, business leaders, and politicians. Cornell's commitment to academic excellence, combined with its vibrant campus life and strong sense of community, make it a highly sought-after institution for students from around the world."